In [1]:
import requests
from requests import Response
from pydantic import BaseModel,Field,field_validator,RootModel
import pandas as pd

youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
if youbike_url.status_code == 200:
    print("下載成功")
else:
    print("下載失敗")

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
        
class Youbike(RootModel):
    root:list[Site]

data = Youbike.model_validate_json(youbike_url.text)
all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df1=df.set_index(keys='行政區域')
df1_sum=df1.groupby(level='行政區域').sum()
df1_sum

下載成功


,總數量,可借數量,可還數量
行政區域,,,
中山區,3819,1087,2708
中正區,3209,1341,1829
信義區,3331,1152,2133
內湖區,3499,1134,2312
北投區,2472,921,1513
南港區,2317,921,1388
士林區,2962,1099,1805
大同區,1630,442,1174
大安區,4987,1827,3146
